In [95]:
import pandas as pd
import math
import numpy as np
import math
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing
from sklearn.preprocessing import normalize # to normalize a matrix
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model, metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from scipy import linalg
from sklearn import metrics
import random
import pyswarm
from sklearn.model_selection import learning_curve # to analyze bias and variance of linear regression
import matplotlib.pyplot as plt # for plotting the curves
from sklearn.svm import SVC
from sklearn.svm import SVR

In [48]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)
# print(pd.Series.sort_values(ratings.movie_id).unique())

13


In [108]:
print("users: ",ratings.user_id.drop_duplicates().count()) #number of users
print("movies: ",ratings.movie_id.drop_duplicates().count()) #number of movies

users:  6078
movies:  976


In [109]:
print("minimum movies rated by a user" ,min(ratings.groupby(["user_id"]).overall.count())) # minimum ratings given by any user
print("maximum movies rated by a user" ,max(ratings.groupby(["user_id"]).overall.count())) # maximum ratings given by any user
print("minimum ratings of a movie" ,min(ratings.groupby(["movie_id"]).overall.count())) # minimum ratings of a movie
print("maximum ratings of a movie" ,max(ratings.groupby(["movie_id"]).overall.count())) # maximum ratings of a movie

minimum movies rated by a user 5
maximum movies rated by a user 237
minimum ratings of a movie 1
maximum ratings of a movie 830


In [110]:
# function to crop the dataset such that ratings given by any user>=ru
def min_user_ratings_criteria(ratings, ru):
    # remove all users who have rated <= ru movies
    temp = ratings.groupby(['user_id']).overall.count().reset_index()
#     print(temp)
    users_to_be_removed = temp[temp.overall < ru].user_id.unique()
    print(len(users_to_be_removed))
    new_ratings = ratings[~ratings['user_id'].isin(users_to_be_removed)]
    print(len(new_ratings.user_id.unique()))
    return new_ratings
#     print(len(users_to_be_removed))
# min_ratings_criteria(10,1)

In [111]:
# function to crop the dataset such that every item is rated by atleast ri users
def min_item_ratings_criteria(ratings, ri):
    # remove all items with number of ratings <= ri
    temp = ratings.groupby(['movie_id']).overall.count().reset_index()
#     print(temp)
    items_to_be_removed = temp[temp.overall < ri].movie_id.unique()
    print(len(items_to_be_removed))
    new_ratings = ratings[~ratings['movie_id'].isin(items_to_be_removed)]
    print(len(new_ratings.movie_id.unique()))
    return new_ratings
# min_item_ratings_criteria(5)

In [112]:
# split the datasets into smaller datasets, such that every dataset contains 'size' number of users
def split_dataset(ratings, size):
        df_group = ratings.groupby(['user_id'])
#         print(type(df_group))
        
        df_group_list = list()
        # converting the group of dataframes into list of dataframe for random shuffling
        for index, item in df_group:
            df_group_list.append(item)
        # randomly shuffling the list
        random.shuffle(df_group_list)
        no_of_datasets = int(len(df_group)/size);
#         print(no_of_datasets)
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        datasets_generated = 0 # datasets that have been generated till now
        for item in df_group_list:
            count+=1
            temp = temp.append(item)
            if count==size and datasets_generated < no_of_datasets-1:
                list_datasets.append(temp)
                print("size = ",temp.shape[0])
#                 print(temp.user_id.unique())
                temp = pd.DataFrame()
                datasets_generated += 1
                count=0
        list_datasets.append(temp) # appending the last dataset
        print("size = ",temp.shape[0])
        return list_datasets
    
# testing above function
# temp = split_dataset(ratings, 1000)
# for item in temp:
#     print(len(item.user_id.unique()))

In [113]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [114]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [160]:
# remove movies with <5 ratings
# rating_dash = min_item_ratings_criteria(ratings, 5)
# remove users who have rated <10 items
# rating_dash = min_user_ratings_criteria(rating_dash, 10)
# print(len(rating_dash.movie_id.unique()))
# list conatining all the datasets
list_datasets = split_dataset(rating_dash,500)
print(rating_dash.shape)

size =  5094
size =  5491
size =  5385
size =  5090
size =  4912
size =  5585
size =  4379
size =  5041
size =  4793
size =  5487
size =  4978
size =  5722
(61957, 8)


In [161]:
# splitting into train and test data
train = list()
test = list()
num_of_datasets = len(list_datasets)
print(num_of_datasets)
for i in range(num_of_datasets):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

12
0
1
2
3
4
5
6
7
8
9
10
11


# Multi-criteria user-user collaborative filtering

In [15]:
# function to find similarity between two users based on Manhattan distance
def manhattan_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += abs(row.criterion1_x - row.criterion1_y) + \
                abs(row.criterion2_x - row.criterion2_y) + \
                abs(row.criterion3_x - row.criterion3_y) + \
                abs(row.criterion4_x - row.criterion4_y) + \
                abs(row.overall_x - row.overall_y)
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# manhattan_similarity(train, 1, 3)

In [16]:
# function to find simialrity between two users based on Euclidean distance
def euclidean_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += math.sqrt((row.criterion1_x - row.criterion1_y)**2 + \
                (row.criterion2_x - row.criterion2_y)**2 + \
                (row.criterion3_x - row.criterion3_y)**2 + \
                (row.criterion4_x - row.criterion4_y)**2) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
# euclidean_similarity(train, 1, 3)

In [17]:
# function to find simialrity between two users based on Chebyshev distance
def chebyshev_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += max(abs(row.criterion1_x - row.criterion1_y), \
                abs(row.criterion2_x - row.criterion2_y), \
                abs(row.criterion3_x - row.criterion3_y), \
                abs(row.criterion4_x - row.criterion4_y), \
                abs(row.overall_x - row.overall_y))
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# chebyshev_similarity(train, 1, 3)

In [18]:
# function to predict the rating given by user to item
# neighbours = -1 implies that consider similarity with all possible users
def predict(df, user, item, similarity, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        sim = similarity(df, user, v) # find appropriate similarity measure between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating

In [19]:
# print(test[0].head(26))

In [20]:
# print(train)
print(predict(train[0], 1, 879, euclidean_similarity))
print(predict(train[0], 1, 879, manhattan_similarity))
print(predict(train[0], 1, 879, chebyshev_similarity))

0
0
0


In [117]:
# function to calculate the mean average error between two vectors
def mean_average_error(list1, list2):
    sum1=0
    for a,b in zip(list1,list2):
        if(b==0):            # to handle the unpredicted ratings
            continue
        sum1 += abs(a-b)
    error = sum1/len(list1)
    return error
mean_average_error([1,2,3],[10,20,30])

18.0

In [118]:
# function to replace entries in a list with zero/one based on some threshold
def replace_with_zero_one(list1, threshold):
    length = len(list1)
    rlist = list()
    i = 0
    while i < length:
        if(list1[i]==0):   # to handle the unpredicted ratings
            list1[i]=-1
            continue
        if(list1[i]>threshold):
            rlist.append(1)
        else:
            rlist.append(0)
        i += 1
    return rlist

In [119]:
# function to calculate the precision between two lists
# list2 is the predicted list
def precision(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fp = 0 # false positives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==1 and a==0):
            fp+=1
    precision = tp/(tp + fp)
    return precision

# precision([20,5,14], [1,16,20], 13)

In [120]:
# function to calculate the recall between two lists
# list2 is the predicted list
def recall(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fn = 0 # false negatives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==0 and a==1):
            fn += 1
#     print("tp = ",tp)
#     print("fn = ",fn)
    recall = tp/(tp + fn)
    return recall

# recall([20,15,14], [1,16,15], 13)

In [121]:
# function to calculate F1-score between two lists
def f1_score(list1, list2, threshold):
    p = precision(list1, list2, threshold)
    r = recall(list1, list2, threshold)
    f1 = (2*p*r)/(p+r)
    return f1

In [162]:
# predicting on our training dataset
predicted = list() # for storing the predicted ratings
# appending 7 empty lists
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1
unpredicted_count = list()
# print(predicted1)

In [27]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    cnt = 0
    for index, row in test[i].iterrows():
        predicted_rating = predict(train[i], row.user_id, row.movie_id,chebyshev_similarity)
#         print(predicted_rating)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
        print(predicted_rating)
    print("predicted %d",i)
    unpredicted_count.append(cnt)

6.0
0
9.34528605963
4.04347826087
11.25
We cannot predict
0
0
0
13.0
9.0
9.84158415842
10.0794775293
12.1232032854
11.9938650307
10.3560797529
12.2122356495
0
0
5.0
10.9189189189
10.5
10.2988854286
We cannot predict
0
10.0769230769
4.83495145631
11.2195121951
0
8.5
11.0324324324
7.90909090909
10.1956521739
1.0
9.10160275661
0
8.95625635809
11.4613402062
9.82075471698
11.8722891566
12.5613577023
11.0
10.0
11.4285714286
6.89156626506
8.95652173913
7.95652173913
9.42857142857
11.6071428571
11.2
5.1186440678
We cannot predict
0
11.132078505
9.203125
7.0487804878
9.05515451375
7.42857142857
7.0
11.887822226
6.76923076923
4.85714285714
11.0801409071
11.9465747519
8.1948376353
10.5135135135
10.0
0
5.36666666667
9.0
6.2
9.66666666667
10.4
8.2
7.0
13.0
4.875
0
12.0
12.1208791209
11.5591127155
0
4.0
7.74747474747
6.0
0
12.4412269004
11.1258709309
10.2
11.0
9.58536585366
11.6379444589
5.79560602623
8.175066313
10.4482315113
10.063469109
8.35709220283
9.52525914504
11.8188861219
10.3054791546
9.24

KeyboardInterrupt: 

In [28]:
# predicting on our training dataset1
print("test-size = ", test[1].shape[0])
predicted[1] = list() # for storing the predicted ratings
i=0
for index, row in test[1].iterrows():
    predicted_rating = predict(train[1], row.user_id, row.movie_id,euclidean_similarity, 20)
    print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[1].append(predicted_rating)
print(predicted[1])

test-size =  3646
0
1
2
3
4
5
6
Continuing
7
8
9


KeyboardInterrupt: 

In [29]:
# print("unpredicted = ",unpredicted_count[1])
print(len(predicted[1]))
threshold = 7
ttest = test[1].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[1], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[1]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

9
mae =  0.00686817891691
precision =  0.8571428571428571
recall =  0.75
f1-score =  0.7999999999999999


# Singular Value Decomposition (SVD)

In [30]:
# function to convert data into mxn matrix m -> number of users, n -> number of movies
# the matrix is normalized to take care of missing ratings
# after normalizations missing ratings can be safely assumed to be zero
# df dataframe should contain columns as 'user_id', 'movie_id', 'rate'
def convert_to_normalized_matrix(df, rating_column):
    no_of_movies = len(ratings.movie_id.unique())
    no_of_users = len(ratings.user_id.unique())
    matrix = np.zeros((no_of_users, no_of_movies))
    for index, row in df.iterrows():
        matrix[row.user_id-1][row.movie_id-1] = row[rating_column]
#     print("completed")
    return matrix,normalize_matrix(matrix)

# function to normalize a matrix by subtracting the average rating of the user
def normalize_matrix(matrix):
#     matrix[matrix == 0] = np.nan
#     matrix = np.ma.array(matrix, mask=np.isnan(matrix))

    # normalize the matrix
    normalized_matrix = np.zeros((len(matrix), len(matrix[0])))
    i=0
#     print(matrix)
    num_users = len(matrix)
    while i < num_users:
        baseline = np.sum(matrix[i])
        baseline /= np.count_nonzero(matrix[i])
        # subtract baseline value from all non-zero ratings
        normalized_matrix[i][:] = [x-baseline if x>0 else x for x in matrix[i]]
        i+=1
    return normalized_matrix
    
    
# g,h = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', 'overall']])
# print(h)

In [31]:
# function to apply SVD on normalized user-item matrix
def svd(ratings, rating_column):
    original,X = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', rating_column]], rating_column)
    print("matrix created, now decomposing it")
    u,s,v = np.linalg.svd(X) # Singular Value Decomposition
    return original,u,s,v

In [32]:
# function to crop the decomposed matrices to top-k latent features
def crop_matrices(u, s, v, k):
    # calculate average rating of the user according to 'matrix'
    cropped_u = u[:,:k] # crop matrix u
    cropped_s = s[:k] # crop array s
    cropped_v = v[:k,:] # crop matrix v
    return cropped_u, cropped_s, cropped_v
# crop_matrices(u,s,v,15)

In [33]:
# to predict the rating given by 'user' to 'item' based on SVD
# u, s, v are the decomposed matrices
# matrix is the parent matrix that is decomposed
def predict_svd(matrix, u, s, v, user, item):
    # calculate average rating of 'user' according to 'matrix'
    baseline = np.sum(matrix[user-1])
    baseline /= np.count_nonzero(matrix[user-1])
#     print("average -> ", baseline)
    p = u[user-1] # latent-features values of the user
    # s is the weight of latent-features
    q = np.transpose(v)[item-1]
    
    return baseline+sum(p*s*q)
    
# matrix,u,s,v = svd(ratings,'overall')
# print("completed")
# a,b,c = crop_matrices(u,s,v,15)
# print("completed")
# predict_svd(matrix,a,b,c,1,1)

In [34]:
# creating the decomposed matrices
matrix,u,s,v = svd(train[0], 'overall')

C:\Users\Garvit\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it


In [35]:
# croppping the matrices to appropriate no. of latent-features
a,b,c = crop_matrices(u,s,v,15)

In [36]:
# testing the svd implementation
# fingers crossed
# predicting on our training dataset1
print("test-size = ", test[0].shape[0])
predicted[0] = list() # for storing the predicted ratings
i=0
for index, row in test[0].iterrows():
    predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
#     print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[0].append(predicted_rating)
# print(predicted[0])

test-size =  3433


In [37]:
# creating empty lists for storing the predicted values
predicted = list() # for storing the predicted ratings
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1

In [38]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    matrix,u,s,v = svd(train[i], 'overall')
    a,b,c = crop_matrices(u,s,v,15)
    for index, row in test[i].iterrows():
        predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
#         print(predicted_rating)
    print("predicted ",i)
#     unpredicted_count.append(cnt)

C:\Users\Garvit\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it
predicted  0
matrix created, now decomposing it
predicted  1
matrix created, now decomposing it
predicted  2
matrix created, now decomposing it
predicted  3
matrix created, now decomposing it
predicted  4
matrix created, now decomposing it
predicted  5


In [143]:
# function for evaluating the model
def evaluate_model(predicted, test):
    print(len(predicted))
    threshold = 7
    ttest = test.overall
    list1 = replace_with_zero_one(ttest.tolist(), threshold)
    list2 = replace_with_zero_one(predicted, threshold)
    # mean average error
    print("mae = ",mean_average_error(ttest, predicted))
    # precision
    print("precision = ",precision(list1, list2, threshold))
    # recall
    print("recall = ",recall(list1, list2, threshold))
    # f1 score
    print("f1-score = ",f1_score(list1, list2, threshold))

# Particle Swarm Optimization (PSO)
## (using the pyswarm library)

In [133]:
# defining the fitness funcion to be optimised by pso
# weights is the list of weights that are to be calculated by PSO
# 'user' is the user for which we want to predict the weights
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    ans = 0
    for index, row in user.iterrows():
        ans += (w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4 - row.overall)**2
    return ans

In [156]:
# function for predicting the label according to our weights got using pso
def predict_pso(xopt, test):
    predicted_list = list()
    w1 = xopt[0]
    w2 = xopt[1]
    w3 = xopt[2]
    w4 = xopt[3]
    for index, row in test.iterrows():
        prediction = w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4
        predicted_list.append(prediction)
    return predicted_list

In [40]:
# Anubhav's implementation of PSO
def pso(fitness_func, lbound, ubound, swarm_size=20, max_iter=40, dimensions=4, phip=0.5, phig=0.5, weight=0.017):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]
            
            
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/10)
        v_max.append((ubound[i]-lbound[i])/10)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*phip*(x_pbest[i][j]-x[i][j]) + r2*phig*(x_gbest[j]-x[i][j])
                
#                 if v[i][j] > v_max[j]:
#                     v[i][j] = v_max[j]
                
#                 if v[i][j] < v_min[j]:
#                     v[i][j] = v_min[j]


                # Kumud's code
                #checking the limit of velocity, if goes beyond then sack it
                if v[i][j] > v_max:
                    vel[i][j] = (vel_max / abs(vel[i][j])*vel[i][j])
                                 
                                 
                
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [176]:
# applying the particle-swarm-optimization
# xopt = the optimal input values
# fopt = the optimal objective value
df = list_datasets[0]
user_list = df.groupby(['user_id'])
lower_bound = [0, 0, 0, 0]
upper_bound = [255, 255, 255, 255]
mae = 0
count = 0
for index, item in user_list:
    # split item in train and test data
    train, test = train_test_split(item, test_size = 0.3, random_state = 1601)
    user = train
    # library implemenatation
#     xopt, fopt = pyswarm.pso(fitness_func, lower_bound, upper_bound)
    # Anubhav's implementation
    xopt, fopt = pso(fitness_func, lower_bound, upper_bound)
    print(xopt)
    print(fopt)
    # predict the values on the test dataset
    predicted_list = predict_pso(xopt, test)
#     print("test shape ", test.shape[0])
    # calculate the mean average error
#     print("checking --> ",test.overall.shape[1])
#     print("debug = ",len(predicted_list))
#     print("debug = ", len(test.overall))
    error = metrics.mean_absolute_error(test.overall.tolist(), predicted_list)
    accuracy = metrics.r2_score(test.overall.tolist(), predicted_list)
    print("mae = ",error)
    mae += error
    count += 1
    if(count==20):
        break

[0, 0, 0, 0]
264
mae =  5.5
[0, 0, 0, 0]
306
mae =  12.5
[0, 63.90047939062116, 0, 16.663029575979127]
2749424.80397
mae =  901.719768288
[0, 0, 42.18954725523602, 0.6950620000336839]
459901.78912
mae =  148.181318394
[0, 0, 0, 0]
402
mae =  13.0
[0, 0, 0, 0]
171
mae =  12.5
[0, 0, 19.409921877282674, 0]
353437.001777
mae =  146.612708352
[0, 0, 0, 0]
492
mae =  12.0
[0, 0, 0, 0]
393
mae =  10.0
[0, 0, 0, 0]
1161
mae =  9.8
[0, 0, 0, 0]
603
mae =  8.5
[0, 0, 0, 0]
492
mae =  6.5
[0, 0, 0, 0]
351
mae =  5.66666666667
[0, 0, 0, 0]
371
mae =  5.5
[0, 0, 0, 0]
486
mae =  11.0
[0, 0, 0, 0]
460
mae =  11.0
[0, 0, 0, 0]
557
mae =  12.0


KeyboardInterrupt: 

In [1]:
print("mae = ",mae/count)

NameError: name 'mae' is not defined

# Linear Regression
## (using library provided by sklearn)

In [123]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [155]:
# applying linear regression on list_datasets
# plotting style
# plt.style.use('seaborn-whitegrid')
print(len(list_datasets))
for i in range(len(list_datasets)):
    df = list_datasets[i]
    df_group = df.groupby(['user_id'])
    count = 0
    avg_error = 0
#     print("mae = ",avg_error/len(list_datasets[i].user_id.unique()))
    flag = 0
    # for plotting graph
#     fig = plt.figure()
#     ax = plt.axes()
    for key,item in df_group:
        # splitting the item in train and test data
        train, test = train_test_split(item, test_size = 0.2, random_state=1212)
        train_X = train[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        train_y = train.overall
        test_X = test[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        test_y = test.overall
        # create linear regression object
        clf = linear_model.LinearRegression(normalize=True)
        # train the model using the training sets
        clf.fit(train_X, train_y)
        print("coefficients: ", clf.coef_)
        print("intercept: ", clf.intercept_)
        prediction = clf.predict(test_X)
        st=0
        # checking the accuracy
        score = clf.score(test_X, test_y)
        
#         print(score)
        
        if(score<0.80 and flag==0):
            count+=1
#             flag=1
            # plotting the learning curve
#             r,s,t = learning_curve(linear_model.LinearRegression(), item[['criterion1', 'criterion2', 'criterion3', 'criterion4']], item.overall)
#             plt.plot(r,s)
#             plt = plot_learning_curve(linear_model.LinearRegression(),"My curve" ,item[['criterion1', 'criterion2', 'criterion3', 'criterion4']], item.overall)    
#             plt.plot(r,t)
#             plt.show()
    print("total = ",len(df_group))
    print("below average = ", count)

6
coefficients:  [-0.34631411  0.84856579  0.55399426 -0.05624594]
intercept:  -3.5527136788e-15
coefficients:  [ 0.15489743  0.66526449  0.25696451  0.14350125]
intercept:  -1.96747825339
coefficients:  [ 0.45454545  1.09090909 -0.45454545 -0.90909091]
intercept:  10.6363636364
coefficients:  [  7.07967718e-15   1.00000000e+00   1.00000000e+00  -1.00000000e+00]
intercept:  4.4408920985e-15
coefficients:  [ 0.24093503  0.24068961  0.25894227  0.25943309]
intercept:  0.0
coefficients:  [  9.00000000e+00   3.00000000e+00  -6.00000000e+00  -1.47017673e-15]
intercept:  -65.0
coefficients:  [-0.38461538  0.43589744  0.43589744  0.43589744]
intercept:  0.974358974359
coefficients:  [ 0.56403956 -0.14100989 -0.32422523  0.98560646]
intercept:  -1.42156689723
coefficients:  [ 0.94 -0.06 -0.06  0.12]
intercept:  0.78
coefficients:  [ 0.4679213   0.4679213   0.15397776 -0.15397776]
intercept:  0.834046193328
coefficients:  [ 0.34103798  0.0189539   0.62917905  0.01000744]
intercept:  0.020740536

intercept:  2.12326325339
coefficients:  [-2.  -6.   6.5 -2. ]
intercept:  48.0
coefficients:  [  2.54762308e-01   4.35550643e-01   1.24337226e-04   2.66927843e-01]
intercept:  0.12320597194
coefficients:  [-0.11855979  0.04059971  0.31470593  0.55599441]
intercept:  0.725499187829
coefficients:  [ 0.33333333  0.          0.33333333  0.33333333]
intercept:  1.7763568394e-15
coefficients:  [ 0.52800467  0.14941486  0.34228805  0.07374369]
intercept:  -0.61920807612
coefficients:  [-0.12499302  1.16711114 -0.16018919  0.14631039]
intercept:  -0.870867076236
coefficients:  [ 0.55974977  0.36526714  0.01404449  0.21681775]
intercept:  -1.8595561621
coefficients:  [ 0.35795124  0.33616315  0.35795124  0.09111486]
intercept:  -1.14544397747
coefficients:  [ 0.4765625   0.4375     -0.47916667  0.33333333]
intercept:  2.34375
coefficients:  [ 0.55726759 -0.59013867  0.62300976 -0.59013867]
intercept:  12.0
coefficients:  [ 0.20534992  0.45546194  0.26433175  0.07878128]
intercept:  -0.14844881

intercept:  -12.6736144008
coefficients:  [-1.1875  0.3125  1.4375 -0.0625]
intercept:  8.25
coefficients:  [  2.58769292e-01   6.01924233e-01   2.49949890e-01  -6.01322910e-04]
intercept:  -0.627781118461
coefficients:  [ 0.  0.  0.  0.]
intercept:  13.0
coefficients:  [  1.99024352e+00  -1.95129566e-03  -1.01951296e+00   4.48798002e-02]
intercept:  -0.0721979394318
coefficients:  [ 5.10294118  0.56862745  0.39705882 -1.96568627]
intercept:  -32.2009803922
coefficients:  [  4.44444444e-01  -2.22222222e-01   7.77777778e-01   1.57513350e-16]
intercept:  -3.5527136788e-15
coefficients:  [ 0.38461538  1.          0.23076923 -0.5       ]
intercept:  -1.53846153846
coefficients:  [ -3.02464809e-16  -6.10774227e-17   5.00000000e-01   5.00000000e-01]
intercept:  2.6645352591e-15
coefficients:  [ 0.30569719  0.9252607   0.18256083 -0.03594051]
intercept:  -4.90851671168
coefficients:  [-0.65829217 -0.75102064  2.01156251 -1.07077397]
intercept:  19.3796214349
coefficients:  [ 0.47115385  0.086

coefficients:  [ 0.41998363  0.21008413  0.29603803  0.08808925]
intercept:  -0.196528032393
coefficients:  [ 1.34615385  0.61538462 -0.75       -0.05769231]
intercept:  -2.11538461538
coefficients:  [ 0.24314963  0.07956178  0.62592365  0.03881876]
intercept:  0.319196428571
coefficients:  [ 0.52891807  0.26409331  0.30678456 -0.09033935]
intercept:  -0.145046151063
coefficients:  [  2.00000000e+00   1.00000000e+00  -6.67479072e-16  -1.00000000e+00]
intercept:  -10.0
coefficients:  [ 0.31553022  0.28074496  0.33415431  0.38510072]
intercept:  -3.89071836647
coefficients:  [ 0.31733525  0.39713772  0.15603102  0.06430975]
intercept:  -0.207120645735
coefficients:  [ 0.7468785   0.57958799 -0.57553072  0.40824021]
intercept:  -1.48969973498
coefficients:  [ 0.3877551   0.34693878  0.30612245 -0.34693878]
intercept:  4.02040816327
coefficients:  [ 0.39960855 -0.14255423  0.1562551   1.01728919]
intercept:  -3.07731202088
coefficients:  [-0.00964207  0.78911614 -0.09379109  0.31731191]
in

intercept:  -7.34357923497
coefficients:  [ 0.71103263  0.24183642  0.13123584 -0.04268002]
intercept:  -0.0804329511453
coefficients:  [ 0.47490347  0.25868726 -0.05019305  0.47490347]
intercept:  -1.89961389961
coefficients:  [ 0.19736842 -0.10526316 -0.03508772 -0.39473684]
intercept:  16.8421052632
coefficients:  [  5.00000000e-01   3.38670215e-16   5.00000000e-01  -5.71757768e-17]
intercept:  -2.6645352591e-15
coefficients:  [ 0.35377149  0.09100191  0.4016978   0.2404634 ]
intercept:  -0.722267333352
coefficients:  [ 0.34433176  0.33884412  0.395596   -0.02577105]
intercept:  -0.383509308141
coefficients:  [ 0.16129032  0.07419355  0.24516129  0.53870968]
intercept:  -0.729032258065
coefficients:  [  4.33371577e-01   4.01127006e-01   1.91313015e-15   4.28710576e-01]
intercept:  -2.55963691991
coefficients:  [  7.40740741e-02   1.66666667e-01   2.77777778e-01  -9.81904886e-17]
intercept:  6.25925925926
coefficients:  [ 0.25    0.25    0.25    0.1875]
intercept:  0.5625
coefficient

...

intercept:  5.72566374863
coefficients:  [  1.00000000e+00  -9.63517828e-18  -7.57034150e-15   1.46190399e-14]
intercept:  -1.7763568394e-15
coefficients:  [ 1.23440135  0.38560989  0.15064643  0.23496346]
intercept:  -12.6874648679
coefficients:  [ -3.95591105e-17  -8.55945200e-16   1.00000000e+00   1.15355040e-15]
intercept:  -5.3290705182e-15
coefficients:  [-0.02794035  0.41532086 -0.15445033  0.70188925]
intercept:  1.13369707438
coefficients:  [ 0.37872892  0.64332036  0.11932555 -0.33592737]
intercept:  0.194552529183
coefficients:  [ 0.70410237  0.44716235 -0.31495983  0.20181951]
intercept:  -3.19145519326
coefficients:  [ 0.45937463  0.37812389  0.24375222 -0.08125074]
intercept:  3.5527136788e-15
coefficients:  [-2.97289973  1.51355014  9.48644986 -6.02710027]
intercept:  -15.972899729
coefficients:  [ 0.39614417  0.57119618 -0.03832879  0.48590251]
intercept:  -4.82644421078
coefficients:  [-0.81229947  0.38449198  0.63409091  0.57513369]
intercept:  2.93355614973
coefficie

coefficients:  [-0.28947368  1.89473684  0.0877193  -0.53508772]
intercept:  -0.157894736842
coefficients:  [ 0.00167416  0.28853978  0.01789017  0.13128541]
intercept:  6.1659366544
coefficients:  [-0.32312925 -0.01020408  1.01020408  0.00340136]
intercept:  4.84353741497
coefficients:  [ 0.2112547   0.3303982   0.36533434  0.6191435 ]
intercept:  -5.13335837716
coefficients:  [-0.05165394  0.3634436   0.55725191  0.23146735]
intercept:  -1.07616624258
coefficients:  [ 1.18047605  0.93416692 -1.20488099  0.29511901]
intercept:  -3.09761976499
coefficients:  [-0.1557377   0.04918033  1.5        -0.33606557]
intercept:  -1.75409836066
coefficients:  [ 0.36317136 -0.15920716 -0.75421995  1.31841432]
intercept:  2.62289002558
coefficients:  [ 0.0761499   0.45054784  0.65664719  0.12754338]
intercept:  -3.53977724785
coefficients:  [-0.04659761  0.02074664  0.74418465  0.18900997]
intercept:  1.09392868904
coefficients:  [-0.56774194  0.49677419  0.43225806  0.66236559]
intercept:  -0.2129

intercept:  -1.06581410364e-14
coefficients:  [ 0.65412876 -0.73069216  0.80725556  0.26930784]
intercept:  -1.42108547152e-14
coefficients:  [ 3.75 -2.   -0.25 -0.75]
intercept:  3.25
coefficients:  [-1.12004389  1.20043889  0.63767387  0.32158002]
intercept:  -0.0396488854423
coefficients:  [-0.06896552  0.68965517  1.10344828 -0.79310345]
intercept:  0.896551724138
coefficients:  [ 0.28786792  0.29054273  0.16809263  0.34951528]
intercept:  -0.553926024708
coefficients:  [ 0.88031514  0.26749096  0.03532116  0.02812123]
intercept:  -2.74623041098
coefficients:  [ -1.21135263e-16   6.66666667e-01   0.00000000e+00  -3.58546380e-17]
intercept:  3.0
coefficients:  [ 0.56697557  0.17186184  0.03454086  0.23083404]
intercept:  -0.145745577085
coefficients:  [-0.63964746  1.45168814  0.3580339   0.02816271]
intercept:  -0.487525423729
coefficients:  [ 0.4469697   0.31818182  0.11363636  0.32575758]
intercept:  -2.78787878788
coefficients:  [ 0.3 -1.5 -1.1  2. ]
intercept:  14.8
coefficient

coefficients:  [-1.07241637  0.14483273  2.00135631  0.22131802]
intercept:  -3.31977026652
coefficients:  [  1.82208706e-15   1.00000000e+00  -5.38788593e-16  -2.14276148e-15]
intercept:  1.7763568394e-14
coefficients:  [  4.00000000e+00  -1.68180538e-15   1.00000000e+00  -4.00000000e+00]
intercept:  -5.3290705182e-15
coefficients:  [-0.2787234   1.6606383  -0.70425532  0.64361702]
intercept:  -5.16489361702
coefficients:  [ 0.77141919 -0.11343035  0.3071452   0.35600394]
intercept:  -4.17479369196
coefficients:  [  6.66666667e-01   1.66666667e-01   1.66666667e-01  -5.72642901e-17]
intercept:  0.0
coefficients:  [ 1.         -0.7826087  -0.32608696  3.32608696]
intercept:  -29.9347826087
coefficients:  [ 0.29569468  0.14350363 -0.08330075  1.12206197]
intercept:  -6.41608303563
coefficients:  [  8.73429184e-16   1.78358209e+00   2.16417910e-01  -3.29850746e-01]
intercept:  -8.37164179104
coefficients:  [ 3.  1. -2. -1.]
intercept:  4.4408920985e-15
coefficients:  [ 0.3010002   0.53480

intercept:  -1.46559633028
coefficients:  [ 1.11111111 -0.09259259 -0.22222222 -0.09259259]
intercept:  3.88888888889
coefficients:  [-0.32098765  2.39506173 -2.33950617  0.66049383]
intercept:  5.52469135802
coefficients:  [ 0.34280324  0.38608895  0.10487413  0.38287252]
intercept:  -2.49763716212
coefficients:  [ -2.00000000e+00   3.00000000e+00  -1.80159230e-15   5.80015459e-16]
intercept:  2.6645352591e-14
coefficients:  [  2.67195767e-01   2.40740741e-01   6.22124383e-17   4.81481481e-01]
intercept:  -0.113756613757
coefficients:  [-0.01247576  0.56035688  1.02100971 -0.60664227]
intercept:  0.149194928057
coefficients:  [  1.00000000e+00   1.70569267e-16   2.38796974e-16   2.98496218e-16]
intercept:  0.0
coefficients:  [-18.625   0.375  14.5     0.125]
intercept:  41.625
coefficients:  [ 1.24206697 -0.24667454 -0.06243738  0.06944527]
intercept:  -0.075618257913
coefficients:  [ 0.30989737  0.04600684  0.32204789  0.36805474]
intercept:  -0.598088946561
coefficients:  [ 0.25  0.

In [163]:
# function to predict weights of a particular user using Linear Regression
# it also returns the trained classifier
def predict_weights(df, user):
    user_df = df[df['user_id']==user]
    user_df_X = user_df[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
    user_df_y = user_df.overall
    # create linear regression object
    clf = linear_model.LinearRegression(normalize=True)
    # train the model using the training sets
    clf.fit(user_df_X, user_df_y)
#     prediction = clf.predict(test_X)
    coef = clf.coef_
    coef = [0 if x<0 else x for x in coef]
    return clf
    

In [164]:
# functions for finding similarity between two users based on their preferences to individual criteria
# based on euclidean distance
# where weights is a list of individual weights of user1
def euclidean_similarity_weighted(df, user1, user2, weights):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
    sum1 = 0
    similarity=0
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    for index, row in s.iterrows():
        sum1 += math.sqrt(w1*((row.criterion1_x - row.criterion1_y)**2) + \
                w2*((row.criterion2_x - row.criterion2_y)**2) + \
                w3*((row.criterion3_x - row.criterion3_y)**2) + \
                w4*((row.criterion4_x - row.criterion4_y)**2)) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity

In [165]:
# function to make predictions using some evolutionary method
def predict_evolutionary(df, user, item, similarity = euclidean_similarity_weighted, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    # get the weights preidcted using linear regression
    clf = predict_weights(df, user)
    coef = clf.coef_
    intercept = clf.intercept_
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        
        sim = similarity(df, user, v, coef) # find similarity between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating 

In [146]:
# predicting using the linear regression model
# predicting on our training dataset1
print("test-size = ", test[1].shape[0])
predicted[1] = list() # for storing the predicted ratings
i=0
for index, row in test[1].iterrows():
    predicted_rating = predict_evolutionary(train[1], row.user_id, row.movie_id,euclidean_similarity_weighted, 20)
#     print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[1].append(predicted_rating)
    print("actual: ", row.overall, "predicted: ",predicted_rating)
#     print()
print(predicted[1])

test-size =  3600
[0.18227062084298248, 0, 1.1049528164949924, 0.53050594181903954]
Continuing
[0.18227062084298248, 0, 1.1049528164949924, 0.53050594181903954]
Continuing
[0.17567567567567569, 0.70270270270270252, 0.17567567567567566, 0.29729729729729731]
actual:  12 predicted:  10.1993684065
[0.17567567567567569, 0.70270270270270252, 0.17567567567567566, 0.29729729729729731]
actual:  9 predicted:  1.80444053693
[1.5333349329892125e-16, 0.0, 1.5000000000000004, 0]
actual:  12 predicted:  8.23877983491
[1.5333349329892125e-16, 0.0, 1.5000000000000004, 0]
actual:  9 predicted:  10.612537956
[1.5333349329892125e-16, 0.0, 1.5000000000000004, 0]
actual:  9 predicted:  9.75319340577
[0, 0, 0.90693545018284494, 0]
Continuing
[0, 0, 0.90693545018284494, 0]
actual:  11 predicted:  12.0
[0.51398604817145011, 0.37530125488240712, 0, 0.26947218678047169]
actual:  13 predicted:  10.767167538
[0.51398604817145011, 0.37530125488240712, 0, 0.26947218678047169]


KeyboardInterrupt: 

In [ ]:
# evaluating the model
evaluate_model(predicted[1], test[1])
